# Time-Based Feature Engineering for Early Warning Churn

This notebook engineers time-based and behavioral features designed to
identify churn risk early, before the customer actually churns.

These features will be used in the churn risk modeling stage.

In [2]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 100)

In [3]:
df = pd.read_csv("../data/raw/telco_customer_churn.csv")

df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:
# Clean column names defensively
df.columns = df.columns.str.strip()
df.columns = df.columns.str.replace(" ", "", regex=False)

df.columns.tolist()

['customerID',
 'gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'tenure',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'MonthlyCharges',
 'TotalCharges',
 'Churn']

In [5]:
df["Churn"].value_counts(normalize=True)

Churn
No     0.73463
Yes    0.26537
Name: proportion, dtype: float64

In [6]:
df["tenure_bucket"] = pd.cut(
    df["tenure"],
    bins=[0, 6, 12, 24, 60, np.inf],
    labels=["0-6m", "6-12m", "1-2y", "2-5y", "5y+"]
)

df["tenure_bucket"].value_counts()

tenure_bucket
2-5y     2426
0-6m     1470
5y+      1407
1-2y     1024
6-12m     705
Name: count, dtype: int64

In [7]:
high_charge_threshold = df["MonthlyCharges"].quantile(0.75)

df["high_monthly_charge_flag"] = (
    df["MonthlyCharges"] >= high_charge_threshold
).astype(int)

df["high_monthly_charge_flag"].value_counts()

high_monthly_charge_flag
0    5272
1    1771
Name: count, dtype: int64

In [8]:
high_charge_threshold = df["MonthlyCharges"].quantile(0.75)

df["high_monthly_charge_flag"] = (
    df["MonthlyCharges"] >= high_charge_threshold
).astype(int)

df["high_monthly_charge_flag"].value_counts()

high_monthly_charge_flag
0    5272
1    1771
Name: count, dtype: int64

In [9]:
service_cols = [
    "OnlineSecurity",
    "OnlineBackup",
    "DeviceProtection",
    "TechSupport",
    "StreamingTV",
    "StreamingMovies"
]

In [10]:
df["service_count"] = (df[service_cols] == "Yes").sum(axis=1)

df["service_count"].describe()

count    7043.000000
mean        2.037910
std         1.847682
min         0.000000
25%         0.000000
50%         2.000000
75%         3.000000
max         6.000000
Name: service_count, dtype: float64

## Leakage Check

All engineered features are based on customer attributes and behaviors
available before churn occurs.

No post-churn or outcome-derived information is used.

In [12]:
# Month-to-month contract flag
df["month_to_month_contract"] = (
    df["Contract"] == "Month-to-month"
).astype(int)

df["month_to_month_contract"].value_counts()

month_to_month_contract
1    3875
0    3168
Name: count, dtype: int64

In [13]:
df.columns.tolist()

['customerID',
 'gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'tenure',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'MonthlyCharges',
 'TotalCharges',
 'Churn',
 'tenure_bucket',
 'high_monthly_charge_flag',
 'service_count',
 'month_to_month_contract']

In [14]:
early_warning_features = [
    "tenure",
    "MonthlyCharges",
    "TotalCharges",
    "service_count",
    "high_monthly_charge_flag",
    "month_to_month_contract"
]

df[early_warning_features + ["Churn"]].head()

,tenure,MonthlyCharges,TotalCharges,service_count,high_monthly_charge_flag,month_to_month_contract,Churn
0,1,29.85,29.85,1,0,1,No
1,34,56.95,1889.5,2,0,0,No
2,2,53.85,108.15,2,0,1,Yes
3,45,42.30,1840.75,3,0,0,No
4,2,70.70,151.65,0,0,1,Yes


In [17]:
early_warning_features = [
    col for col in df.columns 
    if col not in ["Churn", "TotalCharges"]
]

df_early_warning = df[early_warning_features]

In [18]:
leakage_columns = [
    "Churn",
    "TotalCharges",
    "clv_score",
    "segment_label"
]

leakage_found = set(leakage_columns).intersection(df_early_warning.columns)
leakage_found

set()

## Leakage Validation

The early-warning feature set was explicitly validated to ensure
that no post-outcome or future-derived variables are present.

Removed variables:
- Churn (target label)
- TotalCharges (cumulative post-tenure signal)

This guarantees the model can be safely used for real-time
early churn risk detection.

In [19]:
df_early_warning.head()
df_early_warning.shape

(7043, 23)